In [1]:
import pandas as pd
import utils
import datetime as dt
from dateutil.parser import *

In [2]:
df_trades = pd.read_pickle("USD_JPY_H4_trades.pkl")

In [4]:
df_trades.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
0,2020-01-02T02:00:00.000000000Z,579,108.671,108.758,108.658,108.742,108.664,108.751,108.652,108.736,...,108.607,108.808,108.586,0.154,1,1,1,108.8234,108.7618,108.9466
4,2020-01-02T18:00:00.000000000Z,1029,108.526,108.572,108.494,108.561,108.520,108.566,108.488,108.511,...,108.212,108.754,108.206,0.536,1,-1,-1,108.1524,108.3668,107.7236
12,2020-01-06T02:00:00.000000000Z,1526,107.950,108.104,107.932,108.058,107.944,108.098,107.925,108.052,...,107.772,108.130,107.764,0.350,1,1,1,108.1650,108.0250,108.4450
14,2020-01-06T10:00:00.000000000Z,1785,107.988,108.164,107.982,108.120,107.982,108.158,107.975,108.114,...,107.926,108.185,107.920,0.253,1,-1,-1,107.8947,107.9959,107.6923
20,2020-01-07T10:00:00.000000000Z,2026,108.466,108.483,108.368,108.438,108.460,108.477,108.361,108.432,...,108.262,108.506,108.255,0.238,-1,1,1,108.5298,108.4346,108.7202


In [5]:
pair = "USD_JPY"

In [6]:
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, "M5"))

In [7]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [8]:
df_trades["time"] = [parse(x) for x in df_trades.time]
df_raw["time"] = [parse(x) for x in df_raw.time]

In [9]:
df_trades["next"] = df_trades["time"].shift(-1)

In [10]:
df_trades["trade_end"] = df_trades.next + dt.timedelta(hours=3, minutes=55)
df_trades['trade_start'] = df_trades.time + dt.timedelta(hours=4)

In [11]:
df_trades[['time', 'next', 'trade_end', 'trade_start']].head()

,time,next,trade_end,trade_start
0,2020-01-02 02:00:00+00:00,2020-01-02 18:00:00+00:00,2020-01-02 21:55:00+00:00,2020-01-02 06:00:00+00:00
4,2020-01-02 18:00:00+00:00,2020-01-06 02:00:00+00:00,2020-01-06 05:55:00+00:00,2020-01-02 22:00:00+00:00
12,2020-01-06 02:00:00+00:00,2020-01-06 10:00:00+00:00,2020-01-06 13:55:00+00:00,2020-01-06 06:00:00+00:00
14,2020-01-06 10:00:00+00:00,2020-01-07 10:00:00+00:00,2020-01-07 13:55:00+00:00,2020-01-06 14:00:00+00:00
20,2020-01-07 10:00:00+00:00,2020-01-07 18:00:00+00:00,2020-01-07 21:55:00+00:00,2020-01-07 14:00:00+00:00


In [12]:
df_trades.dropna(inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [13]:
def signal_text(signal):
    if signal == 1:
        return 'BUY'
    elif signal == -1:
        return 'SELL'
    return 'NONE'

def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    return False

def end_hit_calc(direction, SL, price, start_price):
    delta = price - start_price
    full_delta = start_price - SL
    fraction = abs(delta / full_delta)
    
    if direction == 1 and price >= start_price:
        return fraction
    elif direction == 1 and price < start_price:
        return -fraction
    elif direction == -1 and price <= start_price:
        return fraction
    elif direction == -1 and price > start_price:
        return -fraction
    
    print("ERROR")
    
def process_buy(TP, SL, ask_prices, bid_prices, entry_price):
    for index, price in enumerate(ask_prices):
        if triggered(1, price, entry_price) == True:
            for live_price in bid_prices[index:]:
                if live_price >= TP:
                    return 2.0
                elif live_price <= SL:
                    return -1.0
            return end_hit_calc(1, SL, live_price, entry_price)
    return 0.0

def process_sell(TP, SL, ask_prices, bid_prices, entry_price):
    for index, price in enumerate(bid_prices):
        if triggered(-1, price, entry_price) == True:
            for live_price in ask_prices[index:]:
                if live_price <= TP:
                    return 2.0
                elif live_price >= SL:
                    return -1.0
            return end_hit_calc(-1, SL, live_price, entry_price)
    return 0.0


In [15]:
total = 0
for index, row in df_trades.iterrows():
    m5_data = df_raw[(df_raw.time >= row.trade_start) & (df_raw.time <= row.trade_end)]
    if row.SIGNAL == 1:
        r = process_buy(row.TAKEPROFIT, row.STOPLOSS, m5_data.ask_c.values, m5_data.bid_c.values, row.ENTRY)
        total += r
    else:
        r = process_sell(row.TAKEPROFIT, row.STOPLOSS, m5_data.ask_c.values, m5_data.bid_c.values, row.ENTRY)
        total += r
print(total)

88.64713208280158
